In [1]:
import re
from lxml.html import fromstring
from unidecode import unidecode
from requests import Session

In [14]:
def get_tree_from_url(session, url, cookies={}):
    response = session.get(url, cookies=cookies)
    return fromstring(response.text)


def extract_price(price):
    price = price.replace('/kg', '')
    return int(re.sub(r"\s|R|\$|,", "", price)) / 100


def process_text(text):
    text = unidecode(text)
    text = text.replace(',', '')
    text = text.lower()
    return text

In [3]:
# Global vars
BASE_URL = 'https://www.arasuper.com.br/'
ALL_DEPS = '//ul[@class="menu-principal__items menu-principal__col menu-principal__col--1"]'
DEP_NAME = 'a/span/span'
DEP_URL = 'a/@href'
SUB_DEPS = 'div/div/ul/li' 

PRODUCT_NAME = 'div/div[3]/div/p[@class="item-produto__name"]'
PRODUCT_BRAND = 'div/div[3]/div/p[@class="item-produto__brand"]'
PRICE_ONE = 'div/div[3]/div/p[@class="item-produto__price just-one"]'
PRICE_TWO = 'div/div[3]/div/p[@class="item-produto__price"]'
PRICE_LEVE = 'div/div[3]/div/p[@class="item-produto__price price-leve"]'
PRICE_REGULAR = 'span[@class="item-produto__price-de"]'
PRICE = 'span[@class="item-produto__price-por"]'
PRODUCTS =  '//div[@class="products-list"]/a'

In [4]:
def get_departments(session):
    """
    Extract all departments from BASE_URL
    """
    departments = {}
    tree = get_tree_from_url(session, BASE_URL)
    for dep in tree.xpath(ALL_DEPS)[1]:
        depName = dep.xpath(DEP_NAME)[0].text_content()    
        subDeps = dep.xpath(SUB_DEPS)
        
        if subDeps != []: # Exists subDeps
            for sub in subDeps:
                subcontent = sub.text_content().split()
                if not(subcontent in [['Todos'],['Voltar']]):     # Erro Dept Todos e Voltar
                    subName = sub.xpath(DEP_NAME)[0].text_content()
                    url = BASE_URL + sub.xpath(DEP_URL)[0]
                    departments[url] = depName + ' > ' + subName # Insert into DEPTS TABLE
        else: # Insert DEP
            url = BASE_URL + dep.xpath(DEP_URL)[0]
            departments[url] = depName # Insert into DEPTS TABLE  
    return departments

In [9]:
def get_pages(session, department_url):
    """
    Get all pages from a departmente given an department_url
    """
    i = 1
    while True:
        new_url = department_url + '?page=' + str(i)
        print(new_url)
        tree = get_tree_from_url(session, new_url)
        page_empty = tree.xpath('//div[@class="produto-lista empty-content"]')
        if page_empty != []:
            break
        else:
            yield tree
        i = i + 1

In [6]:
def extract_products(page):
    products_info = []
    for product in page.xpath(PRODUCTS): 
        product_url = BASE_URL + product.xpath('@href')[0]
        product_name = product.xpath(PRODUCT_NAME)[0].text_content()
        product_brand = product.xpath(PRODUCT_BRAND)[0].text_content()
        
        if product.xpath(PRICE_ONE): # One price
            regular_price = None
            price = product.xpath(PRICE_ONE)[0].text_content()
            price = extract_price(price)
            
        elif product.xpath(PRICE_TWO): # Regular Price and Price
            prices = product.xpath(PRICE_TWO)[0]
            regular_price = prices.xpath(PRICE_REGULAR)[0].text_content()
            regular_price = extract_price(regular_price)
            price = prices.xpath(PRICE)[0].text_content()
            price = extract_price(price)
        
        else: # Only price
            prices = product.xpath(PRICE_LEVE)[0]
            regular_price = prices.xpath(PRICE_REGULAR)[0].text_content()
            regular_price = extract_price(regular_price)
            price = prices.xpath(PRICE)[0].text_content()
            price = extract_price(price)

        
        # INSERT PRODUCT IN TABLE
        products_info.append(
            {
                'url': product_url,
                'name': product_name,
                'brand': product_brand,
                'price': price,
                'regular_price': regular_price
            }
        )

    return products_info


In [7]:
session = Session()

In [8]:
for department_url in get_departments(session).keys():
    print(department_url)

https://www.arasuper.com.br/c/massas-resfriadas/580/
https://www.arasuper.com.br/c/torradas/562/
https://www.arasuper.com.br/c/sal/546/
https://www.arasuper.com.br/c/queijo-ralado/530/
https://www.arasuper.com.br/c/oleo/484/
https://www.arasuper.com.br/c/mostarda/481/
https://www.arasuper.com.br/c/polpa-de-tomate/475/
https://www.arasuper.com.br/c/molho-tomate/474/
https://www.arasuper.com.br/c/manteigas/460/
https://www.arasuper.com.br/c/maionese/457/
https://www.arasuper.com.br/c/frutas/453/
https://www.arasuper.com.br/c/leite-de-coco/429/
https://www.arasuper.com.br/c/leite-condensado/425/
https://www.arasuper.com.br/c/massas/421/
https://www.arasuper.com.br/c/granola/398/
https://www.arasuper.com.br/c/geleias/391/
https://www.arasuper.com.br/c/gelatinas/389/
https://www.arasuper.com.br/c/feijao/362/
https://www.arasuper.com.br/c/misturas-prontas/356/
https://www.arasuper.com.br/c/extrato-de-tomate/353/
https://www.arasuper.com.br/c/leite-cremoso/312/
https://www.arasuper.com.br/c/l

In [15]:
    for department_url in get_departments(session).keys():
        print(department_url)
        # Add timer here 
        for page in get_pages(session, department_url):
            products = extract_products(page)
            for i in products:
                print(i)
            print('--------------------------------')

https://www.arasuper.com.br/c/massas-resfriadas/580/
https://www.arasuper.com.br/c/massas-resfriadas/580/?page=1
{'url': 'https://www.arasuper.com.br/p/massa-de-pastel-grande-romanha-500g/16331/', 'name': 'Massa De Pastel Grande Romanha 500g', 'brand': 'ROMANHA', 'price': 8.49, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/massa-de-lasanha-romanha-500g/16324/', 'name': 'Massa de Lasanha Romanha 500g', 'brand': 'ROMANHA', 'price': 9.99, 'regular_price': None}
--------------------------------
https://www.arasuper.com.br/c/massas-resfriadas/580/?page=2
https://www.arasuper.com.br/c/torradas/562/
https://www.arasuper.com.br/c/torradas/562/?page=1
{'url': 'https://www.arasuper.com.br/p/torrada-bauducco-cereale-integral-128g/18849/', 'name': 'Torrada Bauducco Cereale Integral 128G', 'brand': 'BAUDUCCO', 'price': 5.39, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/torrada-marilan-magic-toast-original-150g/1071/', 'name': 'Torrada Marilan Magic Toast Original 15

https://www.arasuper.com.br/c/queijo-ralado/530/
https://www.arasuper.com.br/c/queijo-ralado/530/?page=1
{'url': 'https://www.arasuper.com.br/p/queijo-ralado-mococa-embalagem-40g/7362/', 'name': 'Queijo Ralado Mococa Embalagem 40G', 'brand': 'MOCOCA', 'price': 3.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/queijo-parmesao-ralado-zaeli-pacote-50g/3668/', 'name': 'Queijo Parmesão Ralado Zaeli Pacote 50G', 'brand': 'ZAELI', 'price': 7.79, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/queijo-parmesao-ralado-president-50g/131/', 'name': 'Queijo Parmesão Ralado President 50G', 'brand': 'PRESIDENT', 'price': 6.29, 'regular_price': None}
--------------------------------
https://www.arasuper.com.br/c/queijo-ralado/530/?page=2
https://www.arasuper.com.br/c/oleo/484/
https://www.arasuper.com.br/c/oleo/484/?page=1
{'url': 'https://www.arasuper.com.br/p/mini-milho-delicias-agrucapers-230g/19744/', 'name': 'Mini Milho Delicias Agrucapers 230G  ', 'brand': 'DELICI

https://www.arasuper.com.br/c/manteigas/460/
https://www.arasuper.com.br/c/manteigas/460/?page=1
{'url': 'https://www.arasuper.com.br/p/manteiga-aviacao-com-sal-lata-200g/3716/', 'name': 'Manteiga Aviação Com Sal Lata 200G', 'brand': 'AVIAÇAO', 'price': 20.75, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/manteiga-cabeca-de-touro-lata-200g/1187/', 'name': 'Manteiga Cabeça De Touro Lata 200G', 'brand': 'CABECA DE TOURO', 'price': 18.89, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/manteiga-tourinho-lata-500g/1175/', 'name': 'Manteiga Tourinho Lata 500G', 'brand': 'TOURINHO', 'price': 35.79, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/manteiga-tourinho-com-sal-lata-200g/1176/', 'name': 'Manteiga Tourinho Com Sal Lata 200G', 'brand': 'TOURINHO', 'price': 16.29, 'regular_price': None}
--------------------------------
https://www.arasuper.com.br/c/manteigas/460/?page=2
https://www.arasuper.com.br/c/maionese/457/
https://www.arasuper.com.br/

{'url': 'https://www.arasuper.com.br/p/macarrao-instantaneo-zaeli-lamen-galinha-embalagem-85g/7837/', 'name': 'Macarrão Instantâneo Zaeli Lámen Galinha Embalagem 85G', 'brand': 'ZAELI', 'price': 1.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-instantaneo-zaeli-lamen-legumes-embalagem-85g/7839/', 'name': 'Macarrão Instantâneo Zaeli Lámen Legumes Embalagem 85G', 'brand': 'ZAELI', 'price': 1.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-instantaneo-zaeli-galinha-caipira-pacote-88g/7838/', 'name': 'Macarrão Instantâneo Zaeli Galinha Caipira Pacote 88G', 'brand': 'ZAELI', 'price': 1.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-instantaneo-zaeli-lamen-carne-e-tomate-embalagem-85g/7835/', 'name': 'Macarrão Instantâneo Zaeli Lámen Carne E Tomate Embalagem 85G', 'brand': 'ZAELI', 'price': 1.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-instantaneo-zaeli-lamen-carne-embalagem-85g/

{'url': 'https://www.arasuper.com.br/p/macarrao-de-semola-conchinha-liane-pacote-500g/3737/', 'name': 'Macarrão De Sêmola Conchinha Liane Pacote 500G', 'brand': 'LIANE', 'price': 4.98, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-de-semola-ave-maria-liane-pacote-500g/3735/', 'name': 'Macarrão De Sêmola Ave Maria Liane Pacote 500G', 'brand': 'LIANE', 'price': 4.98, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-liane-aletria-com-semola-500g/3734/', 'name': 'Macarrão Liane Aletria Com Sêmola 500G', 'brand': 'LIANE', 'price': 6.98, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-com-ovos-bavetti-liane-pacote-500g/3736/', 'name': 'Macarrão Com Ovos Bavetti Liane Pacote 500G', 'brand': 'LIANE', 'price': 6.79, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/macarrao-liane-com-semola-tortine-especial-embalagem-500g/3728/', 'name': 'Macarrão Liane Com Sêmola Tortine Especial Embalagem 500G', 'brand': 'LIANE'

...

{'url': 'https://www.arasuper.com.br/p/biscoito-recheado-parati-hot-cracker-presunto-pacote-150g/5654/', 'name': 'Biscoito Recheado Parati Hot Cracker Presunto Pacote 150G', 'brand': 'PARATI', 'price': 9.59, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/biscoito-maizena-piraque-200g/5480/', 'name': 'Biscoito Maizena Piraquê 200G', 'brand': 'PIRAQUE', 'price': 4.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/biscoito-piraque-salgadinho-100g/5469/', 'name': 'Biscoito Piraquê Salgadinho 100G', 'brand': 'PIRAQUE', 'price': 3.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/biscoito-piraque-agua-gergelim-240g/5465/', 'name': 'Biscoito  Piraquê Água Gergelim 240G', 'brand': 'PIRAQUE', 'price': 6.49, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/biscoito-piraque-roladinho-goiaba-75g/5457/', 'name': 'Biscoito Piraquê Roladinho Goiaba 75G', 'brand': 'PIRAQUE', 'price': 4.1, 'regular_price': None}
{'url': 'https://www.arasuper.c

{'url': 'https://www.arasuper.com.br/p/biscoito-cookie-classic-gotas-de-chocolate-nestle-60g/514/', 'name': 'Biscoito Cookie Clássic Gotas De Chocolate Nestle 60G', 'brand': 'NESTLE', 'price': 2.49, 'regular_price': 3.38}
--------------------------------
https://www.arasuper.com.br/c/biscoitos/193/?page=11
https://www.arasuper.com.br/c/chocolates/167/
https://www.arasuper.com.br/c/chocolates/167/?page=1
{'url': 'https://www.arasuper.com.br/p/bombom-nutsbites-chocolate-meio-amargo-sem-gluten-60g/19946/', 'name': 'Bombom Nutsbites Chocolate Meio Amargo Sem Glúten 60G  ', 'brand': 'NUTSBITES', 'price': 8.69, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/bombom-nutsbites-chocolate-branco-sem-gluten-60g/19921/', 'name': 'Bombom Nutsbites Chocolate Branco Sem Glúten 60G ', 'brand': 'NUTSBITES', 'price': 8.69, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/bombom-nutsbites-sem-gluten-chocolate-ao-leite-60g/19922/', 'name': 'Bombom Nutsbites Sem Glúten Chocolate 

{'url': 'https://www.arasuper.com.br/p/chocolate-espirito-cacau-42-cacau-leite-coco-80g/18291/', 'name': 'Chocolate Espirito Cacau 42% Cacau Leite Coco 80G', 'brand': 'ESPIRITO CACAU', 'price': 20.49, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/chocolate-m-m-s-amendoim-80g/16249/', 'name': "Chocolate M&M's Amendoim 80G", 'brand': 'M&M', 'price': 2.97, 'regular_price': 4.99}
{'url': 'https://www.arasuper.com.br/p/chocolate-barra-neugebauer-ao-leite-embalagem-90g/16213/', 'name': 'Chocolate Barra Neugebauer Ao Leite Embalagem 90G', 'brand': 'NEUGEBAUER', 'price': 5.29, 'regular_price': 5.99}
{'url': 'https://www.arasuper.com.br/p/pastilha-chocolate-m-m-s-35g/16215/', 'name': "Pastilha Chocolate M&M's 35G", 'brand': 'M&M', 'price': 3.39, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/chocolate-barra-neugebauer-meio-amargo-40-cacau-embalagem-90g/16217/', 'name': 'Chocolate Barra Neugebauer Meio Amargo 40% Cacau Embalagem 90G', 'brand': 'NEUGEBAUER', 'price'

{'url': 'https://www.arasuper.com.br/p/mini-tubes-m-ms-30g/4039/', 'name': 'Mini Tubes M&Ms 30G', 'brand': 'M&M', 'price': 3.35, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/chocolate-bis-oreo-lacta-embalagem-100-8g/3890/', 'name': 'Chocolate Bis Oreo Lacta Embalagem 100,8G', 'brand': 'BIS', 'price': 8.99, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/chocolate-lacta-bis-xtra-ao-leite-45g/3891/', 'name': 'Chocolate Lacta Bis Xtra Ao Leite 45G', 'brand': 'LACTA', 'price': 4.15, 'regular_price': None}
{'url': 'https://www.arasuper.com.br/p/chocolate-kit-kat-branco-41-5g/3872/', 'name': 'Chocolate Kit Kat Branco 41,5G', 'brand': 'NESTLE', 'price': 2.95, 'regular_price': 3.98}
{'url': 'https://www.arasuper.com.br/p/chocolate-kit-kat-dark-41-5g/3873/', 'name': 'Chocolate Kit Kat Dark 41,5G', 'brand': 'NESTLE', 'price': 2.95, 'regular_price': 3.98}
{'url': 'https://www.arasuper.com.br/p/chocolate-m-ms-45g/3829/', 'name': 'Chocolate M&Ms 45G', 'brand': 'M&M', 

KeyboardInterrupt: 